In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.metrics import accuracy_score
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split

In [ ]:
df=pd.read_csv("/content/wine.csv")
df.info()
df.shape
df.head()

In [ ]:
df1=df.copy()
df2=df.copy()

In [ ]:
#Data Spliting
X=df.iloc[:,1:14]

In [ ]:
# correlation heatmap
f,ax = plt.subplots(figsize=(10,8))
sns.heatmap(X.corr(), annot=True, linewidths =.5, fmt ='.1f',ax=ax)
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

bp = X.copy()
fig, axes = plt.subplots(12, 1, figsize=(60, 35), sharex=False, sharey=False)
data = ["Alcohol", "Malic", "Ash", "Alcalinity", "Magnesium", "Phenols", "Flavanoids", "Nonflavanoids", "Proanthocyanins", "Color", "Hue", "Dilution", "Proline"]

for i, ax in zip(data, axes):
    sns.boxplot(x=i, data=X, palette='crest', ax=ax)

plt.tight_layout(pad=2.0)
plt.show()


In [ ]:
sns.pairplot(X, height=2.5)
plt.show()

In [ ]:
#Standard Scaler
from sklearn.preprocessing import StandardScaler
SS=StandardScaler()
SS_X=SS.fit_transform(X)
SS_X=pd.DataFrame(SS_X)

SS_X.columns=list(X)
SS_X.head()

In [ ]:
# Principal Component Analysis
from sklearn.decomposition import PCA

# Assuming SS_X is your standardized data
pca = PCA(n_components=3)
pc = pca.fit_transform(SS_X)

# Explained Variance Ratio
explained_var_ratio = (pca.explained_variance_ratio_)*100
explained_var_ratio = pd.DataFrame(explained_var_ratio)
print("Explained Variance Ratio:", explained_var_ratio)


pd.DataFrame(pc).head()


In [ ]:
# Principal Component Analysis
from sklearn.decomposition import PCA

# Assuming SS_X is your standardized data
pca = PCA()
pc = pca.fit_transform(SS_X)

# Explained Variance Ratio
explained_var_ratio = (pca.explained_variance_ratio_)*100
explained_var_ratio = pd.DataFrame(explained_var_ratio,columns=["Explained Variance (%)"])
print("Explained Variance Ratio:", explained_var_ratio)


pd.DataFrame(pc).head()

In [ ]:
# Plotting the explained variance ratio
plt.figure(figsize=(10, 6))
sns.scatterplot(x=range(1, len(explained_var_ratio) + 1), y="Explained Variance (%)", data=explained_var_ratio, s=100, color='skyblue', edgecolor='black')
plt.title('Explained Variance Ratio for Each Principal Component')
plt.xlabel('Principal Component')
plt.ylabel('Explained Variance (%)')

# Annotate points with the corresponding explained variance
for i, txt in enumerate(explained_var_ratio.values):
    plt.annotate(f'{txt[0]:.2f}%', (i + 1, txt), ha='center', va='bottom')

plt.show()

In [ ]:
PC_X=pd.DataFrame(pc[:,0:8])
PC_X.head()

In [ ]:
pca_result = pca.fit_transform(PC_X)
# Creating a scatter plot to visualize PCA scores colored by cluster labels
plt.figure(figsize=(10, 6))
scatter = plt.scatter(
    pca_result[:, 0], pca_result[:, 1], c=cluster_labels, cmap='viridis', alpha=0.6
)
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.title('PCA Scores Colored by Agglomerative Clustering Labels')
plt.colorbar(scatter)
plt.show()

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()

In [ ]:
Y = df["Type"]

In [ ]:
# validation set method --> validaton hold-off method
""" but the input PC_X """

training_accuracy = []
test_accuracy = []

for i in range(1,10,1):
    X_train,X_test,Y_train,Y_test  = train_test_split(PC_X,Y, test_size=0.30, random_state=i)
    logreg.fit(X_train,Y_train) # Bo + b1x1 + B2x2
    Y_pred_train = logreg.predict(X_train)
    Y_pred_test = logreg.predict(X_test)
    training_accuracy.append(accuracy_score(Y_train,Y_pred_train).round(3))
    test_accuracy.append(accuracy_score(Y_test,Y_pred_test).round(3))

In [ ]:
print("Average trianing accuray",np.mean(training_accuracy).round(2))
print("Average test accuray",np.mean(test_accuracy).round(2))

In [ ]:
#Agglomeritive Clustering

import scipy.cluster.hierarchy as shc

for methods in ['single','complete','average','ward']:
   plt.figure(figsize =(20, 6))
   plt.title('Visualising the data, Method- {}'.format(methods))
   Dendrogram1 = shc.dendrogram(shc.linkage(PC_X, method = methods,optimal_ordering=False))

In [ ]:
from sklearn.cluster import AgglomerativeClustering
cluster = AgglomerativeClustering(n_clusters=3, affinity='euclidean', linkage='ward')
Y = cluster.fit_predict(PC_X)

Y = pd.DataFrame(Y)
print("No.Of Cluster\n",Y.value_counts())
df1['Agglomerative_clusters'] = Y
df1.head()

In [ ]:
from sklearn.cluster import AgglomerativeClustering
import pandas as pd

# Initialize AgglomerativeClustering with different linkage options
linkage_options = ['ward', 'complete', 'average', 'single']
for linkage in linkage_options:
    cluster = AgglomerativeClustering(n_clusters=3, affinity='euclidean', linkage=linkage)
    Y = cluster.fit_predict(PC_X)
    Y = pd.DataFrame(Y)
    print(f"Linkage: {linkage}")
    print("No. of Samples in Each Cluster\n", Y.value_counts())

    # Store cluster labels in the DataFrame
    df1[f'Agglomerative_clusters_{linkage}'] = Y

# Display the DataFrame
print(df1.head())


In [ ]:
#K Means Method
#Elbow Method
from sklearn.cluster import KMeans
KMeans()

clust = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i,random_state=10)
    kmeans.fit(PC_X)
    clust.append(kmeans.inertia_)

plt.plot(range(1,11),clust,color="blue")
plt.scatter(range(1, 11), clust,color="red")
plt.xlim()
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('inertial values')
plt.show()

In [ ]:
#KMeans Cluster algorithm using K=4
kmeans = KMeans(n_clusters=3,random_state=11)
# Fitting with inputs
Kmeans = kmeans.fit(PC_X)
# Predicting the clusters
cluster_labels = kmeans.predict(PC_X)
cluster_centers = kmeans.cluster_centers_

In [ ]:
# Create a scatterplot to visualize the K-Means clustering results
plt.figure(figsize=(10, 6))
plt.scatter(PC_X.iloc[:,0], PC_X.iloc[:,1], c=cluster_labels, cmap='viridis', marker='o', edgecolor='k')
plt.title('K-Means Clustering')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.legend()
plt.show()

In [ ]:
cluster_labels = pd.DataFrame(cluster_labels)
cluster_labels.value_counts()
df2['Kmeans_clusters'] = cluster_labels
df2.head()

In [ ]:
pca_result = pca.fit_transform(PC_X)
# Create a scatter plot to visualize PCA scores colored by cluster labels
plt.figure(figsize=(10, 6))
scatter = plt.scatter(
    pca_result[:, 0], pca_result[:, 1], c=cluster_labels, cmap='viridis', alpha=0.6
)
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.title('PCA Scores Colored by Agglomerative Clustering Labels')
plt.colorbar(scatter)
plt.show()